# Cost Analysis

## 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## 2. Connect to Databases

In [2]:
#PROCUREMENTDB

proc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## 3. Define Clean up tool

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## 4. Define Tables

In [4]:
# Dictionary
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""
# LP All currencies
all_lp = """
SELECT
	[Isbn]
	,[Currency]
	,[Price]
FROM
	[Isbn].[ListPrice]
"""
# Bibliography
bilblo_sql = """
SELECT Isbn
    , Title
    , Publisher
    , Author
FROM 
    Isbn.Bibliography
"""
# Echange Rate
exchange_rate = """
SELECT 
    name,
    rate
FROM dbo.XChange
"""
#Supplier Mega List
megalist_sql = """
SELECT 
    [ISBN]
    ,[Supplier]
    ,[Publisher]
    ,[Currency]
    ,[ListPrice]
    ,[Discount]
    ,[CostUnitPrice]
    ,[MaxQtyPerOrder]
    ,[ShipmentOrigin]
    ,[UnitShippingCost]
    ,[SeaFreightCost] AS sea_shipping_cost
  FROM 
    [Process].[SupplierMegaList]
"""

#Import supplier Excel
#Import short_discount
#Import restrictions

### 4.1 Import Dictionary and clean it

In [5]:
pub_dict = pd.read_sql(pub_dict, proc_db)
pub_dict.head()

,PublisherLong,PublisherShort
0,Glencoe McGraw Hill,MCGRAW
1,Glencoe McGraw-Hill,MCGRAW
2,McGraw Create,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


In [6]:
pub_dict.columns = map(str.lower, pub_dict.columns)
pub_dict = pub_dict.fillna('N/A')
for col in list(pub_dict.columns):
    pub_dict[col] = pub_dict.apply(lambda x: clean_up(x[col]), axis =1)

pub_dict.head()

,publisherlong,publishershort
0,GLENCOE MCGRAW HILL,MCGRAW
1,GLENCOE MCGRAW-HILL,MCGRAW
2,MCGRAW CREATE,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


### 4.2 Import Bibliography and clean it

In [7]:
# IMport and read biblio
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,Publisher,Author
0,9780000000057,"Text, Music, Context: A Resource Item #: S05...",CPH,LCMS
1,9780000001030,Surgical Procedures (2016),CIMC,MAVCC
2,9780000001061,Anatomy & Physiology Student Edition,CIMC,CIMC
3,9780000001146,Membean Student License,Archer,None
4,9780000002402,None,LWW,None


In [8]:
biblio.columns = map(str.lower, biblio.columns)
biblio = biblio.fillna('N/A')
biblio['isbn'] = biblio['isbn'].astype(str)
for col in list(biblio.columns):
    biblio[col] = biblio.apply(lambda x: clean_up(x[col]), axis =1)
biblio.head()

,isbn,title,publisher,author
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A
4,9780000002402,N/A,LWW,N/A


Now we Merge bibliography with the dictionary

In [9]:
biblio2 = pd.merge(biblio, pub_dict, how='left', left_on = 'publisher', right_on = 'publisherlong' )
biblio2.drop(columns = ['publisherlong'], inplace = True)
biblio2.rename(columns={'publishershort': 'pub'}, inplace = True)
biblio2.head()

,isbn,title,publisher,author,pub
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS,NaN
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC,NaN
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC,NaN
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A,NaN
4,9780000002402,N/A,LWW,N/A,LWW


### 4.3 Import SupplierMegaList and clean it

In [10]:
megalist = pd.read_sql(megalist_sql,proc_db)
megalist.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,MaxQtyPerOrder,ShipmentOrigin,UnitShippingCost,sea_shipping_cost
0,9781138161184,Rajan,T&F,GBP,120.00,0.53,0.00,1000,UK,3.25,4.5
1,9781138161320,Rajan,T&F,GBP,105.00,0.53,0.00,1000,UK,3.25,4.5
2,9780128054185,Alek,ELSEVIER,EUR,28.95,0.51,0.00,1000,UK,3.25,1.0
3,9780521697767,Zookal,Cambridge,AUD,0.00,0.00,48.21,150,AUS,7.48,1.0
4,9780128054208,Alek,ELSEVIER,EUR,143.00,0.51,0.00,1000,UK,3.25,1.0


In [11]:
megalist.columns = map(str.lower, megalist.columns)
megalist = megalist.fillna('N/A')
megalist['isbn'] = megalist['isbn'].astype(str)
for col in ['isbn', 'supplier', 'publisher', 'currency', 'shipmentorigin']:
    megalist[col] = megalist.apply(lambda x: clean_up(x[col]), axis =1)
megalist.head()

,isbn,supplier,publisher,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost
0,9781138161184,RAJAN,T&F,GBP,120.00,0.53,0.00,1000,UK,3.25,4.5
1,9781138161320,RAJAN,T&F,GBP,105.00,0.53,0.00,1000,UK,3.25,4.5
2,9780128054185,ALEK,ELSEVIER,EUR,28.95,0.51,0.00,1000,UK,3.25,1
3,9780521697767,ZOOKAL,CAMBRIDGE,AUD,0.00,0.00,48.21,150,AUS,7.48,1
4,9780128054208,ALEK,ELSEVIER,EUR,143.00,0.51,0.00,1000,UK,3.25,1


In [12]:
megalist_pub = pd.merge(megalist, biblio2, how='left', on = 'isbn' )
megalist_pub.head()

,isbn,supplier,publisher_x,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost,title,publisher_y,author,pub
0,9781138161184,RAJAN,T&F,GBP,120.00,0.53,0.00,1000,UK,3.25,4.5,N/A,T&F,N/A,T&F
1,9781138161320,RAJAN,T&F,GBP,105.00,0.53,0.00,1000,UK,3.25,4.5,N/A,T&F,N/A,T&F
2,9780128054185,ALEK,ELSEVIER,EUR,28.95,0.51,0.00,1000,UK,3.25,1,PRESENTING AN EFFECTIVE AND DYNAMIC TECHNICAL ...,ELSEVIER,N/A,ELSEVIER
3,9780521697767,ZOOKAL,CAMBRIDGE,AUD,0.00,0.00,48.21,150,AUS,7.48,1,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
4,9780128054208,ALEK,ELSEVIER,EUR,143.00,0.51,0.00,1000,UK,3.25,1,N/A,ELSEVIER,N/A,ELSEVIER


In [13]:
#cleaning pub name
megalist_pub['pub'] = np.where(
    megalist_pub['pub'] == 'N/A'
    , megalist_pub['publisher_x']
    , megalist_pub['pub'])
    
# centralizing price in one column
megalist_pub['price'] = np.where(megalist_pub['listprice'] == 0.00, megalist_pub['costunitprice'], megalist_pub['listprice'])

#clean column names
megalist_pub.rename(columns={'unitshippingcost': 'shipping_cost', 'maxqtyperorder': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

#Arrange columns and drop unnecessary columns

megalist_pub = megalist_pub[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost', 'max_qty', 'origin']]

megalist_pub.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9781138161184,N/A,T&F,N/A,RAJAN,GBP,120.00,0.53,3.25,4.5,1000,UK
1,9781138161320,N/A,T&F,N/A,RAJAN,GBP,105.00,0.53,3.25,4.5,1000,UK
2,9780128054185,PRESENTING AN EFFECTIVE AND DYNAMIC TECHNICAL ...,ELSEVIER,N/A,ALEK,EUR,28.95,0.51,3.25,1,1000,UK
3,9780521697767,N/A,UNIVERSITY PRESS,N/A,ZOOKAL,AUD,48.21,0.00,7.48,1,150,AUS
4,9780128054208,N/A,ELSEVIER,N/A,ALEK,EUR,143.00,0.51,3.25,1,1000,UK


### 4.3 Import all_lp and clean it

In [14]:
all_lp = pd.read_sql(all_lp, prod_db)
all_lp.head()

,Isbn,Currency,Price
0,9789633861745,GBP,18.99
1,9789633861851,GBP,48.00
2,9789633863121,GBP,62.00
3,9789633863237,GBP,47.00
4,9789633863275,GBP,23.99


In [15]:
all_lp.columns = map(str.lower, all_lp.columns)
all_lp = all_lp.fillna('N/A')
all_lp['isbn'] = all_lp['isbn'].astype(str)
for col in ['isbn','currency']:
    all_lp[col] = all_lp.apply(lambda x: clean_up(x[col]), axis =1)

all_lp.head()

,isbn,currency,price
0,9789633861745,GBP,18.99
1,9789633861851,GBP,48.00
2,9789633863121,GBP,62.00
3,9789633863237,GBP,47.00
4,9789633863275,GBP,23.99


#### 4.3.1 Get biblio with LP

In [16]:
biblio_lp = pd.merge(all_lp, biblio2, how='left', on = 'isbn' )
biblio_lp.drop(columns = ['pub'], inplace = True)
biblio_lp.head()

,isbn,currency,price,title,publisher,author
0,9789633861745,GBP,18.99,HOW THEY LIVED,N/A,N/A
1,9789633861851,GBP,48.00,EXPANDING INTELLECTUAL PROPERTY,N/A,N/A
2,9789633863121,GBP,62.00,PSYCHOLOGY AND POLITICS,N/A,N/A
3,9789633863237,GBP,47.00,"LVIV - WROCLAW, CITIES IN PARALLEL?",N/A,N/A
4,9789633863275,GBP,23.99,THREE CS OF HIGHER EDUCATION,N/A,N/A


### 4.4 Import Sup_by_pub and clean it

In [17]:
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,ShipmentOrigin,Shipping Cost Sea
0,COINFO,3D TOTAL,AUD,0.22,7.48,10000,0,AUS,1.0
1,COINFO,ABC CLIO,AUD,0.30,7.48,10000,0,AUS,1.0
2,COINFO,AIAA,AUD,0.30,7.48,10000,0,AUS,1.0
3,COINFO,ALIEN & UNWIN,AUD,0.30,7.48,10000,0,AUS,1.0
4,COINFO,Amer Psych Ass.,AUD,0.33,7.48,10000,0,AUS,1.0


In [18]:
sup_pub.columns = map(str.lower, sup_pub.columns)
sup_pub = sup_pub.fillna('N/A')
for col in ['supplier','publisher', 'currency', 'shipmentorigin']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)
sup_pub.head()

,supplier,publisher,currency,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,COINFO,3D TOTAL,AUD,0.22,7.48,10000,0,AUS,1
1,COINFO,ABC CLIO,AUD,0.30,7.48,10000,0,AUS,1
2,COINFO,AIAA,AUD,0.30,7.48,10000,0,AUS,1
3,COINFO,ALIEN & UNWIN,AUD,0.30,7.48,10000,0,AUS,1
4,COINFO,AMER PSYCH ASS.,AUD,0.33,7.48,10000,0,AUS,1


#### 4.4.1 Merge Supplier by Publisher with biblio

In [19]:
sup_pub_biblio = pd.merge(biblio_lp, sup_pub, how='inner', left_on = ['currency', 'publisher'], right_on = ['currency', 'publisher'])
sup_pub_biblio.head()

,isbn,currency,price,title,publisher,author,supplier,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,9789652870001,GBP,90.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
1,9789814774017,GBP,116.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
2,9789814774086,GBP,31.99,CHEMISTRY,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
3,9789814774109,GBP,95.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
4,9789814774178,GBP,28.99,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1


In [20]:

sup_pub_biblio.rename(columns={'publisher': 'pub','shipping cost': 'shipping_cost', 'max quantity': 'max_qty', 'shipmentorigin': 'origin', 'shipping cost sea' : 'sea_shipping_cost'}, inplace = True)

sup_pub_biblio.head()

,isbn,currency,price,title,pub,author,supplier,discount,shipping_cost,max_qty,min,origin,sea_shipping_cost
0,9789652870001,GBP,90.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
1,9789814774017,GBP,116.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
2,9789814774086,GBP,31.99,CHEMISTRY,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
3,9789814774109,GBP,95.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1
4,9789814774178,GBP,28.99,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1


#### 4.5 Import short_disc and clean it

In [21]:
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [22]:
sp_disc.columns = map(str.lower, sp_disc.columns)
sp_disc = sp_disc.fillna('N/A')
sp_disc['isbn'] = sp_disc['isbn'].astype(str)
for col in ['supplier', 'isbn']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)
sp_disc.head()

,supplier,isbn,discount
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


#### 4.5.1 Merge sup_pub_biblio by sp_disc with biblio

In [23]:
sup_biblio_disc = pd.merge(sup_pub_biblio, sp_disc, how='left', on = ['supplier', 'isbn'])
sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y
0,9789652870001,GBP,90.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN
1,9789814774017,GBP,116.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN
2,9789814774086,GBP,31.99,CHEMISTRY,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN
3,9789814774109,GBP,95.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN
4,9789814774178,GBP,28.99,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN


In [24]:
sup_biblio_disc['discount'] = np.where(sup_biblio_disc['discount_y'].isna(), sup_biblio_disc['discount_x'],sup_biblio_disc['discount_y'])


#Drop unneccessary Columns

sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y,discount
0,9789652870001,GBP,90.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN,0.35
1,9789814774017,GBP,116.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN,0.35
2,9789814774086,GBP,31.99,CHEMISTRY,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN,0.35
3,9789814774109,GBP,95.00,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN,0.35
4,9789814774178,GBP,28.99,N/A,T&F,N/A,ABE,0.35,3.25,10000,0,UK,1,NaN,0.35


In [25]:
sup_biblio_disc = sup_biblio_disc[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'sea_shipping_cost', 'max_qty', 'origin']]
sup_biblio_disc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9789652870001,N/A,T&F,N/A,ABE,GBP,90.00,0.35,3.25,1,10000,UK
1,9789814774017,N/A,T&F,N/A,ABE,GBP,116.00,0.35,3.25,1,10000,UK
2,9789814774086,CHEMISTRY,T&F,N/A,ABE,GBP,31.99,0.35,3.25,1,10000,UK
3,9789814774109,N/A,T&F,N/A,ABE,GBP,95.00,0.35,3.25,1,10000,UK
4,9789814774178,N/A,T&F,N/A,ABE,GBP,28.99,0.35,3.25,1,10000,UK


### 4.6 Create table for WC

In [26]:
can_sup = biblio_lp[biblio_lp['currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['discount'] = np.where(can_sup['price']<=100, 0.07,np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14)))
can_sup['max_qty'] = 10000
can_sup['origin'] = 'CAN'
can_sup['shipping_cost'] = 2
can_sup.rename(columns={'publisher': 'pub'}, inplace = True)
can_sup['currency'] = 'CAD'
can_sup['sea_shipping_cost'] = 2

can_sup = can_sup[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]

can_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
12,9789639776296,N/A,N/A,N/A,WC,CAD,29.95,0.07,2,2,10000,CAN
117,9789888083275,N/A,UNIVERSITY PRESS,N/A,WC,CAD,40.00,0.07,2,2,10000,CAN
433,9789812465290,N/A,N/A,N/A,WC,CAD,13.95,0.07,2,2,10000,CAN
439,9789812466129,N/A,N/A,N/A,WC,CAD,11.95,0.07,2,2,10000,CAN
444,9789812466136,N/A,N/A,N/A,WC,CAD,11.95,0.07,2,2,10000,CAN


## 5. Append SupplierMegaLis, PublisherSuppliers and WesterCampus

In [27]:
all_sup = megalist_pub.append([sup_biblio_disc, can_sup], ignore_index= True)

all_sup_biblio = pd.merge(all_sup,biblio[['isbn', 'publisher']] , how='left', on = 'isbn')

all_sup_biblio['pub'] = np.where(all_sup_biblio['pub'].isna(), all_sup_biblio['publisher'], all_sup_biblio['pub'])

all_sup = all_sup_biblio[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]
all_sup['sea_shipping_cost'] = np.where( (all_sup['origin'] == 'US') | (all_sup['origin'] == 'CAN') , all_sup['shipping_cost'], all_sup['sea_shipping_cost'])
all_sup.isna().sum()

isbn                   0
title                166
pub                  166
author               166
supplier               0
currency               0
price                  0
discount               0
shipping_cost          0
sea_shipping_cost      0
max_qty                0
origin                 0
dtype: int64

## 6. Add Fees

In [28]:
#Invoice Fee
all_sup['invoice_fee'] = np.where(     # IF
    all_sup['supplier'].str.contains('ALEK'), 0.005,   #Condition and True
    np.where(  # IF False
        (all_sup['supplier'].str.contains('LAURENTIU')) | (all_sup['supplier'].str.contains('SENAD')),0.02,   #Condition and True
        np.where(  #IF FALSE
            (all_sup['supplier'].str.contains('ARMANDO')) | (all_sup['supplier'].str.contains('FELIPE')),0.01, #Condition and True
            np.where(   #IF FALSE
                (all_sup['supplier'] == 'COINFO'), -1/11, #Condition and True
                0)   # FALSE
                )
                )
                )
# lp_fee

all_sup['lp_fee'] = np.where(
    (all_sup['supplier'].str.contains('BILLSON') & (all_sup['discount'] == 0 )), 0.03,0)

all_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee
0,9781138161184,N/A,T&F,N/A,RAJAN,GBP,120.00,0.53,3.25,4.5,1000,UK,0.000,0.0
1,9781138161320,N/A,T&F,N/A,RAJAN,GBP,105.00,0.53,3.25,4.5,1000,UK,0.000,0.0
2,9780128054185,PRESENTING AN EFFECTIVE AND DYNAMIC TECHNICAL ...,ELSEVIER,N/A,ALEK,EUR,28.95,0.51,3.25,1,1000,UK,0.005,0.0
3,9780521697767,N/A,UNIVERSITY PRESS,N/A,ZOOKAL,AUD,48.21,0.00,7.48,1,150,AUS,0.000,0.0
4,9780128054208,N/A,ELSEVIER,N/A,ALEK,EUR,143.00,0.51,3.25,1,1000,UK,0.005,0.0


## 7. import and Merge exchage rate with all suppliers

In [29]:
exch_df = pd.read_sql(exchange_rate,proc_db)
exch_df.rename(columns={'name': 'currency'}, inplace = True)
all_sup_exc = pd.merge(all_sup, exch_df, how='left', on = 'currency')
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate
0,9781138161184,N/A,T&F,N/A,RAJAN,GBP,120.00,0.53,3.25,4.5,1000,UK,0.000,0.0,1.272783
1,9781138161320,N/A,T&F,N/A,RAJAN,GBP,105.00,0.53,3.25,4.5,1000,UK,0.000,0.0,1.272783
2,9780128054185,PRESENTING AN EFFECTIVE AND DYNAMIC TECHNICAL ...,ELSEVIER,N/A,ALEK,EUR,28.95,0.51,3.25,1,1000,UK,0.005,0.0,1.159172
3,9780521697767,N/A,UNIVERSITY PRESS,N/A,ZOOKAL,AUD,48.21,0.00,7.48,1,150,AUS,0.000,0.0,0.713700
4,9780128054208,N/A,ELSEVIER,N/A,ALEK,EUR,143.00,0.51,3.25,1,1000,UK,0.005,0.0,1.159172


## 8. Cost Before exchange rate and landed cost

In [30]:
all_sup_exc['inv_price_before_exc'] = all_sup_exc['price']*(1-all_sup_exc['discount']+all_sup_exc['lp_fee']*(1+all_sup_exc['invoice_fee']))
all_sup_exc['regular_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['shipping_cost']
all_sup_exc['sea_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['sea_shipping_cost']
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
0,9781138161184,N/A,T&F,N/A,RAJAN,GBP,120.00,0.53,3.25,4.5,1000,UK,0.000,0.0,1.272783,56.4000,75.034950,76.285
1,9781138161320,N/A,T&F,N/A,RAJAN,GBP,105.00,0.53,3.25,4.5,1000,UK,0.000,0.0,1.272783,49.3500,66.061831,67.3118
2,9780128054185,PRESENTING AN EFFECTIVE AND DYNAMIC TECHNICAL ...,ELSEVIER,N/A,ALEK,EUR,28.95,0.51,3.25,1,1000,UK,0.005,0.0,1.159172,14.1855,19.693428,17.4434
3,9780521697767,N/A,UNIVERSITY PRESS,N/A,ZOOKAL,AUD,48.21,0.00,7.48,1,150,AUS,0.000,0.0,0.713700,48.2100,41.887474,35.4075
4,9780128054208,N/A,ELSEVIER,N/A,ALEK,EUR,143.00,0.51,3.25,1,1000,UK,0.005,0.0,1.159172,70.0700,84.473153,82.2232


## 9. Restrictions

In [31]:
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest.rename(columns={'isbn13': 'isbn'}, inplace = True)
rest['isbn'] = rest['isbn'].astype(str)
for col in ['isbn', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn','vendor']]
rest.head()

,isbn,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [32]:
# restricted to all suppliers
risky = rest[rest['vendor'] == 'RISKY']
all_sup_exc = all_sup_exc[~all_sup_exc['isbn'].isin(list(risky['isbn']))]

In [33]:
# restriction per supplier
all_rest = pd.merge(all_sup_exc, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn', 'vendor'] )
all_rest[~(all_rest['vendor'].isna())].head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost,vendor
54122,9781138954366,PRIMATE BEHAVIORAL ECOLOGY,T&F,KAREN B. STRIER,UT,CAD,161.00,0.00,1.27,1.27,1000,CAN,0.000,0.0,0.747468,161.0000,121.612347,121.612,UT
72368,9781506333939,INTRODUCTION TO CRIMINOLOGY INTERACTIVE EBOOK ...,SAGE,FRANK E. HAGAN,ABE,GBP,54.00,0.40,3.25,1,500,UK,0.000,0.0,1.272783,32.4000,44.488163,42.2382,ABE
72491,9781506337982,DISCOUNTED BUNDLE,SAGE,CHRISTOPHER P. NECK,ALEK,GBP,102.00,0.43,3.25,1,1000,UK,0.005,0.0,1.272783,58.1400,77.249592,74.9996,ALEK
72508,9781506340012,INTRODUCTION TO TEACHING INTERACTIVE EBOOK,SAGE,GENE E. (ERWIN) HALL; LINDA F. (FAY) QUINN; DO...,ALEK,GBP,46.99,0.43,3.25,1,30,UK,0.005,0.0,1.272783,26.7843,37.340596,35.0906,ALEK
97813,9781452225371,MAKING SENSE OF THE SOCIAL WORLD INTERACTIVE E...,SAGE,DANIEL F. CHAMBLISS; RUSSELL K. SCHUTT,ABE,GBP,51.00,0.40,3.25,1,500,UK,0.000,0.0,1.272783,30.6000,42.197154,39.9472,ABE


## 10. Getting all posible orders for cost_analysis

In [34]:
all_rest.drop(
    all_rest[
        (all_rest['supplier'] == all_rest['vendor'])
        ].index, inplace = True
)
all_rest.drop(columns=['vendor'], inplace = True)
all_rest = all_rest.sort_values(by=['regular_landed_cost'])
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2651277,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.5041
1976400,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,1000,US,0.005,0.0,1.272783,0.0798,0.601568,0.601568
3845381,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.63
3851594,9781259922428,MEASUREMENT BY THE PHYSICAL EDUCATOR: WHY AND HOW,MCGRAW,DAVID R MILLER,RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.63
2144550,9781101950821,N/A,RANDOM HOUSE,N/A,ALEK,USD,0.95,0.66,0.5,0.5,1500,US,0.005,0.0,1.000000,0.3230,0.823000,0.823


In [35]:
all_rest = all_rest.fillna('N/A')
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2651277,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
1976400,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,1000,US,0.005,0.0,1.272783,0.0798,0.601568,0.601568
3845381,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
3851594,9781259922428,MEASUREMENT BY THE PHYSICAL EDUCATOR: WHY AND HOW,MCGRAW,DAVID R MILLER,RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2144550,9781101950821,N/A,RANDOM HOUSE,N/A,ALEK,USD,0.95,0.66,0.5,0.5,1500,US,0.005,0.0,1.000000,0.3230,0.823000,0.823000


## Extra: Getting suppliers from UK, CAN, US

In [36]:
first_filter = all_rest[
    (all_rest['origin'] == 'UK')
    | (all_rest['origin'] == 'US')
    | (all_rest['origin'] == 'CAN')
    ]

first_filter.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2651277,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
1976400,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,1000,US,0.005,0.0,1.272783,0.0798,0.601568,0.601568
3845381,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
3851594,9781259922428,MEASUREMENT BY THE PHYSICAL EDUCATOR: WHY AND HOW,MCGRAW,DAVID R MILLER,RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2144550,9781101950821,N/A,RANDOM HOUSE,N/A,ALEK,USD,0.95,0.66,0.5,0.5,1500,US,0.005,0.0,1.000000,0.3230,0.823000,0.823000


In [37]:
isbn_list = list(pd.read_excel('important_files/list_cost_analysis.xlsx',  sheet_name='list')['Isbn'].astype(str).astype(str))

In [38]:
ocenia_asia_europe = all_rest[~all_rest['origin'].isin(['CAN', 'US', 'CANADA'])]
ocenia_asia_europe.to_csv('retail/ocenia_asia_europe.csv', index= False)
uk_us_can = first_filter[first_filter['isbn'].isin(isbn_list)]
uk_us_can.to_csv('retail/uk_us_can.csv', index= False)
all_suppliers = all_rest[all_rest['isbn'].isin(isbn_list)]
all_suppliers.to_csv('retail/all_suppliers.csv', index= False)
# elsevier = all_rest[all_rest['pub'].str.contains('ELSEVIER')]
# elsevier.to_csv('retail/elsevier.csv', index= False)
# elsevier_cengage = all_rest[all_rest['pub'].str.contains('CENG') | all_rest['pub'].str.contains('ELSE')]
gardner = all_rest[all_rest['isbn'].isin(isbn_list)].drop_duplicates(subset ="isbn", keep = False) 
gardner.to_csv('retail/gardner.csv', index= False)

In [39]:
# all_rest.to_csv('important_files/python/ca_results.csv', index= False)


